In [1]:
from humanitarian_management_system.models.resourceAllocator import ResourceAllocator
from humanitarian_management_system.models.resourceAdder import ResourceAdder
from humanitarian_management_system.models.resourceReport import ResourceReport

from pathlib import Path
import pandas as pd

In [3]:
resource_stock_csv_path = "../data/resourceStock.csv"
resource_allocation_csv_path = "../data/resourceAllocation.csv"
resource_unallocated_stock_csv_path = "../data/resourceUnallocatedStock.csv"

totalResources_df = pd.read_csv(resource_stock_csv_path)
resourceCampMap = pd.read_csv(resource_allocation_csv_path)
unallocResources_df = pd.read_csv(resource_unallocated_stock_csv_path)

stockManualResources = totalResources_df ### the total numbers in this are out of date and we need to update them using resourceCampMap

print(stockManualResources)
print(resourceCampMap)

    resourceID                      name    qty  priorityLvl
0            1                  Blankets  10084            2
1            2                  Clothing   2101            1
2            3                      Toys     40            3
3            4                  Medicine    102            1
4            5                  Vitamins     83            2
5            6                      Food    102            1
6            7                    Snacks     65            3
7            8                Toiletries     83            2
8            9                     Water    113            1
9           10             Baby Supplies    102            1
10          11  General Tools & Supplies     96            3
    resourceID  campID   qty
0            1       3    78
1            2       3   808
2            3       3     0
3            4       3    39
4            5       3    32
5            6       3    39
6            7       3    25
7            8       3    32
8      

In [35]:
joined_df = pd.merge(stockManualResources, resourceCampMap, on='resourceID', how='inner')
print(joined_df)
resource_sum = joined_df.groupby('resourceID').agg({
                'name': 'first',  # Keeps the first name for each group
                'qty': 'sum',  # Sums the qty for each camp >> we ignore the 'total' column in stockManualResources
                'priorityLvl': 'first',  
            }).reset_index()

KeyError: "['total'] not found in axis"

In [21]:
report_instance = ResourceReport()
resource_camp = report_instance.resource_report_camp()
print(resource_camp)

joined_df_unalloc_camp = pd.merge(unallocResources_df, resource_camp, on='resourceID', how='inner')
print(joined_df_unalloc_camp)




campID                                            3     5     6    9    11
resourceID name                     priorityLvl                           
1          Blankets                 2             78  6903  1551  776  776
2          Clothing                 1            808     6   963  162  162
3          Toys                     3             25    20    10    5    5
4          Medicine                 1             39    31    16    8    8
5          Vitamins                 2             32    26    13    6    6
6          Food                     1             39    31    16    8    8
7          Snacks                   3             25    20    10    5    5
8          Toiletries               2             32    26    13    6    6
9          Water                    1             43    35    17    9    9
10         Baby Supplies            1             39    31    16    8    8
11         General Tools & Supplies 3             37    30    15    7    7
    resourceID           

In [18]:
temp = joined_df_unalloc_camp['unallocTotal'].copy()
joined_df_unalloc_camp['unallocTotal'],  joined_df_unalloc_camp['priorityLvl'] = joined_df_unalloc_camp['priorityLvl'], temp
print(joined_df_unalloc_camp)

    resourceID                      name  unallocTotal  priorityLvl    3  \
0            1                  Blankets             2            0   78   
1            2                  Clothing             1            0  808   
2            3                      Toys             3           25   25   
3            4                  Medicine             1            0   39   
4            5                  Vitamins             2            0   32   
5            6                      Food             1            0   39   
6            7                    Snacks             3            0   25   
7            8                Toiletries             2            0   32   
8            9                     Water             1            0   43   
9           10             Baby Supplies             1            0   39   
10          11  General Tools & Supplies             3            0   37   

       5     6    9   11  
0   6903  1551  776  776  
1      6   963  162  162  
2     

In [19]:
joined_df_unalloc_camp.rename(columns={'unallocTotal': 'priorityLvl', 'priorityLvl': 'unallocTotal'}, inplace=True)
print(joined_df_unalloc_camp)

    resourceID                      name  priorityLvl  unallocTotal    3  \
0            1                  Blankets            2             0   78   
1            2                  Clothing            1             0  808   
2            3                      Toys            3            25   25   
3            4                  Medicine            1             0   39   
4            5                  Vitamins            2             0   32   
5            6                      Food            1             0   39   
6            7                    Snacks            3             0   25   
7            8                Toiletries            2             0   32   
8            9                     Water            1             0   43   
9           10             Baby Supplies            1             0   39   
10          11  General Tools & Supplies            3             0   37   

       5     6    9   11  
0   6903  1551  776  776  
1      6   963  162  162  
2     

In [32]:
report_instance = ResourceReport()
resource_camp = report_instance.resource_report_camp_vs_unallocated()
print(resource_camp)

resource_camp = joined_df.pivot_table(index=['resourceID', 'name', 'priorityLvl'], columns='campID', values='qty', aggfunc='sum').sort_index(level=0)
joined_df_unalloc_camp = pd.merge(unallocResources_df, resource_camp, on='resourceID', how='inner')
#print(joined_df_unalloc_camp)

    resourceID                      name  priorityLvl  unallocTotal    3  \
0            1                  Blankets            0             2   78   
1            2                  Clothing            0             1  808   
2            3                      Toys            0             3   25   
3            4                  Medicine            0             1   39   
4            5                  Vitamins            0             2   32   
5            6                      Food            0             1   39   
6            7                    Snacks            0             3   25   
7            8                Toiletries            0             2   32   
8            9                     Water            0             1   43   
9           10             Baby Supplies            0             1   39   
10          11  General Tools & Supplies            0             3   37   

       5     6    9   11  
0   6903  1551  776  776  
1      6   963  162  162  
2     